In [ ]:
```xml
<VSCode.Cell language="markdown">
# SPR 2026 - TF-IDF + SGDClassifier (TRATADO)

**Versão tratada do 2º melhor modelo (0.75019)**

**Tratamento aplicado:**
- ✅ Normalização de texto (lowercase, acentos)
- ✅ Remoção de stop words (preservando termos médicos)
- ✅ Preservação de termos BI-RADS
- ✅ Limpeza de caracteres especiais

**Hipótese:** Laudos muito parecidos → tratamento cuidadoso pode ajudar a capturar diferenças sutis.

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. **IMPORTANTE:** Execute "Run All" após commit
---
</VSCode.Cell>
<VSCode.Cell language="python">
# =============================================================================
# SPR 2026 - TFIDF + SGDCLASSIFIER (VERSÃO TRATADA)
# =============================================================================
# Teste de hipótese: tratamento de texto melhora performance?
# Baseline: 0.75019 (sem tratamento)
# =============================================================================

import numpy as np
import pandas as pd
import re
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - TF-IDF + SGDClassifier (TRATADO)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)

# =============================================================================
# FUNÇÕES DE TRATAMENTO DE TEXTO
# =============================================================================

# Stop words em português (excluindo termos médicos importantes)
STOP_WORDS_PT = {
    'a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo',
    'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele',
    'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em',
    'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estas',
    'este', 'estes', 'eu', 'foi', 'fomos', 'for', 'foram', 'há', 'isso',
    'isto', 'já', 'lhe', 'lhes', 'lo', 'mas', 'me', 'mesmo', 'meu', 'meus',
    'minha', 'minhas', 'muito', 'na', 'nas', 'nem', 'no', 'nos', 'nossa',
    'nossas', 'nosso', 'nossos', 'num', 'numa', 'nuns', 'o', 'os', 'ou',
    'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando',
    'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sem', 'seu', 'seus',
    'só', 'sua', 'suas', 'também', 'te', 'tem', 'tendo', 'tenho', 'ter',
    'teu', 'teus', 'ti', 'tive', 'tivemos', 'tiveram', 'tu', 'tua', 'tuas',
    'um', 'uma', 'umas', 'uns', 'você', 'vocês', 'vos'
}

# Termos BI-RADS importantes (NÃO remover)
BIRADS_TERMS = {
    'birads', 'bi-rads', 'categoria', 'calcificacao', 'calcificacoes',
    'nodulo', 'nodulos', 'massa', 'massas', 'assimetria', 'assimetrias',
    'distorcao', 'densidade', 'benigno', 'benigna', 'maligno', 'maligna',
    'suspeito', 'suspeita', 'provavelmente', 'tipicamente', 'achado',
    'achados', 'mama', 'mamas', 'mamografia', 'ultrassom', 'ecografia',
    'axila', 'axilar', 'linfonodo', 'linfonodos', 'pele', 'mamilo',
    'areola', 'parenquima', 'fibroglandular', 'adiposo', 'heterogeneo',
    'homogeneo', 'denso', 'densa', 'espiculado', 'circunscrito', 'irregular',
    'oval', 'redondo', 'lobulado', 'microlobulado', 'obscurecido',
    'parcialmente', 'totalmente', 'grosseiras', 'finas', 'pleomorficas',
    'amorfas', 'puntiformes', 'lineares', 'ramificadas', 'segmentar',
    'regional', 'difuso', 'agrupadas', 'clusters', 'estavel', 'novo',
    'aumentou', 'diminuiu', 'inalterado', 'recomenda', 'biopsia',
    'controle', 'seguimento', 'rotina', 'complementar', 'negativo',
    'positivo', 'inconclusivo'
}

def remove_accents(text):
    """Remove acentos do texto."""
    nfkd = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in nfkd if not unicodedata.combining(c))

def preprocess_text(text):
    """
    Pré-processa texto de laudo de mamografia.
    - Lowercase
    - Remove acentos
    - Remove caracteres especiais (mantém hífen para BI-RADS)
    - Remove stop words (preservando termos médicos)
    """
    if pd.isna(text):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Remove acentos
    text = remove_accents(text)
    
    # Normaliza BI-RADS
    text = re.sub(r'bi[\s-]*rads?', 'birads', text)
    
    # Remove caracteres especiais (mantém letras, números e espaços)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # Remove números isolados (mas mantém medidas como contexto)
    text = re.sub(r'\b\d+\b', '', text)
    
    # Remove espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stop words (preservando termos BI-RADS)
    words = text.split()
    words = [w for w in words if w not in STOP_WORDS_PT or w in BIRADS_TERMS]
    
    return ' '.join(words)

# ==== CARREGAR DADOS ====
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# ==== TRATAMENTO DE TEXTO ====
print("\n[2/5] Aplicando tratamento de texto...")
train['report_treated'] = train['report'].apply(preprocess_text)
test['report_treated'] = test['report'].apply(preprocess_text)

# Mostrar exemplo
print("\n    Exemplo de tratamento:")
print(f"    Original: {train['report'].iloc[0][:100]}...")
print(f"    Tratado:  {train['report_treated'].iloc[0][:100]}...")

# ==== TF-IDF ====
print("\n[3/5] Aplicando TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train = tfidf.fit_transform(train['report_treated'])
X_test = tfidf.transform(test['report_treated'])
y_train = train['target'].values
print(f"    TF-IDF shape: {X_train.shape}")

# ==== SGDCLASSIFIER ====
print("\n[4/5] Treinando SGDClassifier...")
model = SGDClassifier(
    loss='log_loss',
    penalty='l2',
    alpha=1e-4,
    max_iter=1000,
    tol=1e-3,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1,
    early_stopping=True,
    validation_fraction=0.1
)
model.fit(X_train, y_train)
print("    ✓ Modelo treinado!")

# ==== SUBMISSÃO ====
print("\n[5/5] Gerando submissão...")
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())
</VSCode.Cell>
```